In [1]:
# 📦 Required imports
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.preprocessing import StandardScaler
import re
import numpy as np

# 📂 Step 1: Load the data
csv_file = "hgcal_gcn_training_layer1_combined.csv"
df = pd.read_csv(csv_file)

# 🧹 Step 2: Extract wafer names and coordinates (from uX_vY)
feature_cols = df.columns[1:]  # Skip lumi column
uv_coords = []

for col in feature_cols:
    match = re.match(r"u(-?\d+)_v(-?\d+)", col)
    if match:
        u, v = int(match.group(1)), int(match.group(2))
        uv_coords.append((u, v))
    else:
        raise ValueError(f"Invalid wafer column format: {col}")

# 🧱 Step 3: Convert coordinates to 2D positions
coord_tensor = torch.tensor(uv_coords, dtype=torch.float)

# 🧮 Step 4: Compute edge index using k-nearest neighbors (e.g., k=6)
from torch_geometric.nn import knn_graph

edge_index = knn_graph(coord_tensor, k=6)

# 📏 Step 5: Normalize the features (occupancy)
features = df[feature_cols].values
scaler = StandardScaler()
normalized_features = scaler.fit_transform(features)

# 📦 Step 6: Create list of PyG Data objects (one per lumi)
data_list = []

for i in range(len(df)):
    x = torch.tensor(normalized_features[i], dtype=torch.float).unsqueeze(1)  # shape (N, 1)
    data = Data(x=x, edge_index=edge_index)
    data_list.append(data)

# 🧳 Optional: Create DataLoader
batch_size = 8
loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)

print(f"Number of graph samples: {len(data_list)}")
print(f"Number of nodes per graph: {x.shape[0]}")


Number of graph samples: 100
Number of nodes per graph: 306


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class GCNEncoder(nn.Module):
    def __init__(self, in_channels, hidden_channels, latent_dim):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, latent_dim)

    def forward(self, x, edge_index):
        # Encode node features into latent space
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class GCNDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_channels, out_channels):
        super(GCNDecoder, self).__init__()
        self.conv1 = GCNConv(latent_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, z, edge_index):
        # Decode latent representation back to original features
        z = self.conv1(z, edge_index)
        z = F.relu(z)
        z = self.conv2(z, edge_index)
        return z

class GCNAutoEncoder(nn.Module):
    def __init__(self, in_channels=1, hidden_channels=32, latent_dim=16):
        super(GCNAutoEncoder, self).__init__()
        self.encoder = GCNEncoder(in_channels, hidden_channels, latent_dim)
        self.decoder = GCNDecoder(latent_dim, hidden_channels, in_channels)

    def forward(self, x, edge_index, batch):
        # Encode to latent space
        z = self.encoder(x, edge_index)
        
        # Optionally do pooling or anomaly detection here using `z` and `batch`
        # For now, we just reconstruct node-wise features
        out = self.decoder(z, edge_index)
        return out


In [3]:
# 🔥 Step 7: Setup training - loss function and optimizer
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = GCNAutoEncoder(in_channels=1, hidden_channels=32, latent_dim=16).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()


Using device: cpu


In [4]:
# 🔥 Step 8: Training loop (simple version)

num_epochs = 10
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)
        loss = criterion(out, batch.x)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch.num_graphs
    avg_loss = total_loss / len(data_list)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.6f}")


Epoch 1/10, Loss: 0.853485
Epoch 2/10, Loss: 0.814506
Epoch 3/10, Loss: 0.811804
Epoch 4/10, Loss: 0.811379
Epoch 5/10, Loss: 0.812889
Epoch 6/10, Loss: 0.811750
Epoch 7/10, Loss: 0.810910
Epoch 8/10, Loss: 0.814934
Epoch 9/10, Loss: 0.810457
Epoch 10/10, Loss: 0.810137
